# J Labs

### The Tower of Hanoi

### (1 of 9) The Problem
The Tower of Hanoi problem is to move a set of n
different-sized disks from one peg to another,
moving one disk at a time, using an intermediate
peg if necessary.  At all times no larger disk may
sit on top of a smaller disk.

For example, moving 3 disks from peg 0 to peg 1
can be done as follows:

<code> ```move disk 0 from peg 0 to peg 1```</code><br>
<code> ```move disk 1 from peg 0 to peg 2```</code><br>
<code> ```move disk 0 from peg 1 to peg 2```</code><br>
<code> ```move disk 2 from peg 0 to peg 1```</code><br>
<code> ```move disk 0 from peg 2 to peg 0```</code><br>
<code> ```move disk 1 from peg 2 to peg 1```</code><br>
<code> ```move disk 0 from peg 0 to peg 1```</code><br>

The description of the moves can be shortened if
we observed that in moving a disk from peg A to
peg B, it is always the top disk on peg A that
is moved.  Thus the 3-disk problem can be solved
as follows:

<code> ```0 1```</code><br>
<code> ```0 2```</code><br>
<code> ```1 2```</code><br>
<code> ```0 1```</code><br>
<code> ```2 0```</code><br>
<code> ```2 1```</code><br>
<code> ```0 1```</code><br>

Legend has it that a group of priests has been
solving the 64-disk problem since the beginning
of time, and when they finish, the world will
come to an end.

### (2 of 9) An Initial Solution
There is a simple recursive solution to the n-disk
problem:  First, move disks 0 to n-2 from peg 0 to
peg 2, then move disk n-1 from peg 0 to peg 1,
then move disks 0 to n-2 from peg 2 to peg 1.
If there is just one disk, the one disk can be
moved from peg 0 to peg 1 straightaway.

This can be implemented as a dyadic verb: the right
argument is the number of disks; the left argument
are 3 integers of the pegs (from, to, via).

In [ ]:
H=: 4 : 0
 if. 1>:y do.
  (y,2)$x
 else.
  ((0 2 1{x) H y-1), (2{.x), ((2 1 0{x) H y-1)
 end.
)

In [ ]:
0 1 2 H 3

### (3 of 9) An Initial Solution (ctd)
From the definition of H, it is easy to see that
the n-disk problem requires <:2^n moves.  If n=1,
there is one move (1=<:2^1).  If n=k-1 required
<:2^k-1 moves, then n=k requires the following
numbers of moves:

<code> ```<:2^k-1    ((0 2 1{x) H y-1)```</code><br>
<code> ```1          (2{.x)```</code><br>
<code> ```<:2^k-1    ((2 1 0{x) H y-1)```</code><br>

for a total of <:2^k moves.  A similar argument
shows that the n-disk problem requires <:2^n calls
of the verb H .

In [ ]:
0 1 2 #@H"1 0 i.10

In [ ]:
<: 2 ^ i.10

### (4 of 9) Singly Recursive Solutions
The two recursive steps in H differ only in the
labelling of the pegs. Therefore we can replace
them with a single recursive call, and do two
relabellings of the pegs to get the effect of two
recursions.  That is verb H1 below.

Moreover, in verb H1 the left argument is unchanged
in the recursion; that is, the left argument is
always 0 1 2, and can be elided.

We saw previously that on the n-disk problem the
doubly recursive H required <:2^n calls.  In the
singly recursive H1 and H2, the n-disk problem
requires n calls.  Benchmarks on the 10-disk
problem demonstrate the difference this makes.

In [ ]:
H1=: 4 : 0
 if. 1>:y do.
  (y,2)$0 1{x
 else.
  ({&0 2 1 , 0 1"_ , {&2 1 0) x H1 y-1
 end.
)

In [ ]:
H2=: 3 : 0
 if. 1>:y do.
  i.y,2
 else.
  ({&0 2 1 , 0 1"_ , {&2 1 0) H2 y-1
 end.
)

In [ ]:
0 1 2 (H -: H1)"1 0 i.10

In [ ]:
(0 1 2&H -: H2)"0 i.10

In [ ]:
ts=: 6!:2 , 7!:2@]   NB. time and space

In [ ]:
ts '0 1 2 H 10'

In [ ]:
ts '0 1 2 H1 10'

In [ ]:
ts 'H2 10'

### (5 of 9) Non-Recursive Solutions
We now proceed to derive non-recursive solutions
to the problem.

The results of verbs H, H1, and H2 are rows of the
6-row matrix A (the 6 different ways of moving from
peg i to peg j where i and j can be 0, 1, or 2).
Thus a more efficient representation for the
solutions is to encode the moves as row indices
of A, the integers from 0 to 5.

In [ ]:
A=: 6 2 $ 0 1 0 2 1 0 1 2 2 0 2 1

In [ ]:
A

In [ ]:
A i. H2 1

In [ ]:
A i. H2 2

In [ ]:
A i. H2 3

In [ ]:
A i. H2 4

In [ ]:
A i. H2 5

### (6 of 9) Non-Recursive Solutions (ctd)
Judicious alignment of the indices reveals a pattern:

<code> ```1:   0```</code><br>
<code> ```2: 1 0 5```</code><br>

<code> ```2:   1   0   5```</code><br>
<code> ```3: 0 1 3 0 4 5 0```</code><br>

<code> ```3:   0   1   3   0   4   5   0```</code><br>
<code> ```4: 1 0 5 1 2 3 1 0 5 4 2 5 1 0 5```</code><br>

<code> ```4:   1   0   5   1   2   3   1   0   5   4   2   5   1   0   5```</code><br>
<code> ```5: 0 1 3 0 4 5 0 1 3 2 4 3 0 1 3 0 4 5 0 4 3 2 4 5 0 1 3 0 4 5 0```</code><br>

To get from  A i. H2 n-1  to  A i. H2 n , merge
(intersperse) the result for n with 1 5 2 if n is
even and with 0 3 4 if n is odd.  Thus:

In [ ]:
H3=: 3 : 0
 if. 1>:y do.
  y$0
 else.
  }: , ((2^y-1)$(2|y){1 5 2,:0 3 4) ,. 0,~H3 y-1
 end.
)

In [ ]:
H3 3

In [ ]:
(A i. H2 3) -: H3 3

In [ ]:
(A&i.@H2 -: H3)"0 i.10

### (7 of 9) Non-Recursive Solutions (ctd)
H3 n works by appending an atom to the result of
H3 n-1 ; then stitching a list, repetitions of
1 5 2 or 0 3 4; then ravelling; then dropping the
last (previously appended) atom.

The list of numbers to be stitched (repetitions of
0 3 4 or 1 5 2) depends only on n, and on H3 n-1
not at all.  This suggests a different method of
directing the computation:  Compute the lists xi
of numbers to be stitched for 1, 2, 3, ..., up to n,
and then apply an appropriate verb f between them:

<code> ```xn f ... f x3 f x2 f x1```</code><br>
<code> ```> f&.> / xn ; ... ; x3 ; x2 ; x1```</code><br>

Moreover, we can avoid incorporating into f the
appending and dropping of atoms, if we start out
with an "extra" atom and drop it only at the end:

<code> ```> f&.> / xn ; ... ; x3 ; x2 ; x1```</code><br>
<code> ```}: > g&.> / xn ; ... ; x3 ; x2 ; x1 ; atom```</code><br>

In H4 below, the verb g is ,@,. (ravel stitch).

In [ ]:
H4 =: }: @ > @ (,@,.&.>/) @ (< ,~ 2&^@i. $&.>&|. $&(0 3 4;1 5 2))

In [ ]:
(H3 -: H4)"0 i.10

In [ ]:
arg=: < ,~ 2&^@i. $&.>&|. $&(0 3 4;1 5 2)

In [ ]:
arg 1

In [ ]:
arg 2

In [ ]:
arg 3

In [ ]:
arg 4

In [ ]:
arg 5

### (8 of 9) Non-Recursive Solutions (ctd)
There is another possibility. The pattern in H4 is:

<code> ```}: > g&.> / xn ; ... ; x3 ; x2 ; x1 ; atom```</code><br>

In an intermediate stage, when g is being applied,

<code> ```xi g yi```</code><br>

g "knows" from yi alone what xi has to be:
the length of yi determines the length of xi,
and the leading atom of yi (0 or 1) determines
whether 1 5 2 or 0 3 4 should be ravel-stitched.
This suggests another solution: a monad is applied
n times to the atom 1.

In [ ]:
H5=: [: }: (,@,.~ # $ {. { (1 5 2,:0 3 4)"_)^:(]`1:)

In [ ]:
(H4 -: H5)"0 i.10

In [ ]:
rsa=: ,@,.~ # $ {. { (1 5 2,:0 3 4)"_

In [ ]:
rsa 1

In [ ]:
rsa rsa 1

In [ ]:
rsa rsa rsa 1

In [ ]:
rsa^:(3) 1

In [ ]:
H5 3

### (9 of 9) Conclusion
The Tower of Hanoi problem can be solved in a
variety of ways, with a wide variation in efficiency.

<code> ```H   double recursion```</code><br>
<code> ```H1  single recursion```</code><br>
<code> ```H2  single recursion monad```</code><br>
<code> ```H3  single recursion, atomic representation```</code><br>
<code> ```H4  non-recursive, insert```</code><br>
<code> ```H5  non-recursive, power```</code><br>

In [ ]:
0j8 0 ": ts;._1 '/0 1 2 H  10/0 1 2 H1 10/H2 10/H3 10/H4 10/H5 10'

### End of Lab